In [71]:
import numpy as np
import pandas as pd
import json

from tqdm import tqdm

import time

import pyautogui

from selenium import webdriver

import os
from os.path import exists
from unidecode import unidecode

# Open file to extract list of names

In [1]:
files = ['data/selections/colleges_and_universities.parquet']

df_all = pd.read_parquet(files[0])

df_all = df_all.reset_index(drop=True)
df_all = df_all.drop(df_all.index[df_all['TRANSACTION_DT'] == 'FL'])
df_all = df_all.drop(df_all.index[df_all['TRANSACTION_DT'] == ''])

OCCUPATION_dict = ['professor'] # ['professor', 'faculty', 'SCIENTIST', 'LECTURER']
df_all = df_all[df_all['OCCUPATION'].str.contains("|".join(OCCUPATION_dict), 
                                     case = False)]
        

# Total number of unique names

In [8]:
len(np.unique(df_all['NAME']))

108267

# Clean names
Delete special symbols and various prefixes like Mr, Ms, PhD.
Capitalize everything.

In [76]:
uni_names = np.unique(df_all['NAME'])


def clean(s):
    s = unidecode(s)
    s = s.upper()
    s = s.replace('.','')
    s = s.replace('`','')
    s = s.replace('"','')
    s = s.replace("'",'')
    s = s.replace('*','')
    s = s.replace('?','')
    s = s.replace('/',' ')
    s = s.replace('\\','')
    s = s.split(' ')
    if 'MS' in s:
        s.remove('MS')
    if 'MR' in s:
        s.remove('MR')
    if 'Mr' in s:
        s.remove('Mr')
    if 'MRS' in s:
        s.remove('MRS')
    if 'DR' in s:
        s.remove('DR')
    if 'PHD' in s:
        s.remove('PHD')
    if '' in s:
        s.remove('')
    s = ' '.join(s)
    return(s)

uni_names_clean = [clean(s) for s in uni_names]
uni_names_clean = np.unique(uni_names_clean)
uni_names_clean = uni_names_clean[uni_names_clean != '']

def split_name(s):
    s = s.split(',')
    if len(s) > 1:
        return s[0], s[1][1:]
    else:
        return '', ''
    
uni_names_clean = [split_name(s) for s in uni_names_clean]
uni_names_clean = np.array(uni_names_clean)
uni_names_clean = uni_names_clean[uni_names_clean[:,0]!='', :]

# Numer of unique names after cleaning

In [77]:
uni_names_clean.shape

(99887, 2)

# Launching Chrome driver
You have to donwload chromedriver.exe for your specific version of Chrome (many instructions online). 

In [230]:
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': 'd:\\'}
options.add_experimental_option('prefs', prefs)
   #defined drive location
    
driver = webdriver.Chrome(executable_path="C:/Users/h-desktop/Downloads/chromedriver_win32/chromedriver.exe", chrome_options=options)



<ipython-input-230-43186f31193e>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/h-desktop/Downloads/chromedriver_win32/chromedriver.exe", chrome_options=options)
<ipython-input-230-43186f31193e>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="C:/Users/h-desktop/Downloads/chromedriver_win32/chromedriver.exe", chrome_options=options)


# Query for SCOPUS
We run in a weird way through JS console. This is the only way I could make it work with VPN connected.

In [231]:
def get_auth_data(lname, fname, driver, apikey):
    result = driver.execute_async_script('''
    var done = arguments[0];
    let res;
    fetch("https://api.elsevier.com/content/search/author?query=authlast(''' + lname + ''')%20and%20authfirst(''' + fname + ''')&apiKey=''' + apikey + '''")
         .then(resp=>resp.json())
         .then(data=>{res = done(data)});
    ''')
    return result 

# Loop through the names and retrieve and save JSONs of search results

In [233]:
apikey = ['0fd07fa9c9c4d001ab07aeaab27c1f99', 'bdc326f7799d6022456f4549fbd9f0a6', '72f142ecd004ccb6fe07290c06979708', 'f1f024a0a23126c05ba6b6ef4fddb40e']
# apikey = ['f9b80d824c5f0337ab5e1de00bcafa20', 'cb0ab27c914692912e9bba2245b01c3c', '7ea7ca00ebb9b00b1bfe3846fff8fcd2', '4bd1eda13c0c35fc025b38bbed3b2c99']
apikey = ['a227ce8e04357c18f1ffb80270f07a02',
'5a56c1429405660ade947deb1260a7c6',
'8542d9eec4e3791cbbe8d5043215acb3',
'0f5f55352206e75ff800ef4b47af19fd',
'86ad2910d16cd1946ec1bc2296c68643',
'ccf727f5c8354cfa92ee64395681efdb',
'508410f93da3b8217a8857b97c285df6',
'1dc4643cdc39ab0035c72939c04ae55a',
'66b73b0f7c78970afa644d82bac01d78',
'b5f3405db44bff95a068f5c00ef8a96a']

for ii in tqdm(np.array(range(len(uni_names_clean)))[:]):
    filename = "C:\\data_academics\\" + uni_names_clean[ii,0] + ', ' + uni_names_clean[ii,1] + '.json'
    if not exists(filename):
        x = get_auth_data(uni_names_clean[ii,0], uni_names_clean[ii,1], driver, apikey=apikey[ii % len(apikey)])
        if 'error-response' not in x.keys():
            if 'service-error' not in x.keys():
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(x, f, ensure_ascii=False, indent=4)
#     else:
#         with open(filename, 'r', encoding='utf-8') as f:
#             x = json.load(f)
#             if 'error-response' in x.keys():
#                 x = get_auth_data(uni_names_clean[ii,0], uni_names_clean[ii,1], driver)
#                 with open(filename, 'w', encoding='utf-8') as f:
#                     json.dump(x, f, ensure_ascii=False, indent=4)                


100%|██████████████████████████████████████████████████████████████████████████| 99887/99887 [1:07:41<00:00, 24.59it/s]


In [193]:

for ii in tqdm(range(len(uni_names_clean))):
    filename = "C:\\data_academics\\" + uni_names_clean[ii,0] + ', ' + uni_names_clean[ii,1] + '.json'
    if exists(filename):
        delete = False
        with open(filename, 'r', encoding='utf-8') as f:
            x = json.load(f)
            if 'error-response' in x.keys():
                delete = True
            if 'service-error' in x.keys():
                delete = True
        if delete:
            os.remove(filename)

100%|███████████████████████████████████████████████████████████████████████████| 99887/99887 [10:15<00:00, 162.24it/s]


In [194]:
uni_names_clean[ii,:]

array(['ZYSMAN', 'JOHN'], dtype='<U30')

In [195]:
with open('C:\\data_academics\\BURKHART, HAROLD E.json', 'r', encoding='utf-8') as f:
    x = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\data_academics\\BURKHART, HAROLD E.json'

In [234]:
import glob

In [235]:
files = glob.glob('C:\\data_academics\\*.json')
names = [f.split('\\')[-1][:-5] for f in files]

In [249]:
lookup_subj_area = []
for n in tqdm(names):
    filename = "C:\\data_academics\\" + n + '.json'
    if exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            x = json.load(f)
            if 'search-results' in x.keys():
                if 'error' not in x['search-results']['entry'][0].keys():
                    try:
                        if len(x['search-results']['entry']) == 1:
                            temp = x['search-results']['entry'][0]['subject-area']
                            if type(temp) == dict:
                                lookup_subj_area.append([n,  \
                                                     [[temp['@abbrev'], float(temp['@frequency'])]]])
                            else:
                                lookup_subj_area.append([n,  \
                                                     [[xx['@abbrev'], float(xx['@frequency'])] for xx in temp]])
                    except:
                        pass

100%|██████████████████████████████████████████████████████████████████████████| 83281/83281 [01:01<00:00, 1349.85it/s]


In [248]:
x['search-results']['entry']

[{'@_fa': 'true',
  'affiliation-current': {'affiliation-city': 'Berkeley',
   'affiliation-country': 'United States',
   'affiliation-id': '60025038',
   'affiliation-name': 'University of California, Berkeley',
   'affiliation-url': 'https://api.elsevier.com/content/affiliation/affiliation_id/60025038'},
  'dc:identifier': 'AUTHOR_ID:6602689341',
  'document-count': '44',
  'eid': '9-s2.0-6602689341',
  'link': [{'@_fa': 'true',
    '@href': 'https://api.elsevier.com/content/author/author_id/6602689341',
    '@ref': 'self'},
   {'@_fa': 'true',
    '@href': 'https://api.elsevier.com/content/search/author?query=au-id%286602689341%29',
    '@ref': 'search'},
   {'@_fa': 'true',
    '@href': 'https://www.scopus.com/author/citedby.uri?partnerID=HzOxMe3b&citedAuthorId=6602689341&origin=inward',
    '@ref': 'scopus-citedby'},
   {'@_fa': 'true',
    '@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=6602689341&origin=inward',
    '@ref': 'scopus-author'}],
  'na

In [250]:
for i in range(len(lookup_subj_area)):
    s = 0
    for q in lookup_subj_area[i][1]:
        s += q[1]
    for q in lookup_subj_area[i][1]:
        q[1] /= s
    

In [251]:
import pandas as pd

In [252]:
lookup_subj_area = np.array(lookup_subj_area, dtype=object)

In [253]:
lookup_subj_area_pd = pd.DataFrame(index=lookup_subj_area[:,0])

In [254]:
all_areas = [np.array(i)[:, 0] for i in lookup_subj_area[:,  1]]
all_areas = np.concatenate(all_areas)
all_areas = np.unique(all_areas)

In [255]:
for i in all_areas:
    lookup_subj_area_pd[i] = 0.0

In [256]:
for row in tqdm(lookup_subj_area):
    for q in row[1]:
        lookup_subj_area_pd.loc[row[0]][q[0]] = q[1]

100%|██████████████████████████████████████████████████████████████████████████| 27762/27762 [00:25<00:00, 1097.08it/s]


In [257]:
lookup_subj_area_pd

,AGRI,ARTS,BIOC,BUSI,CENG,CHEM,COMP,DECI,DENT,EART,...,MATH,MEDI,MULT,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE
"A, GILBERT",0.0,0.0,0.000000,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.47619,0.000000,0.000000,0.0
"AADALEN, SHARON P",0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.166667,0.0,0.0,0.666667,0.0,0.00000,0.000000,0.000000,0.0
"AAGAARD, TODD",0.0,0.0,0.000000,0.157895,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.736842,0.0
"AAKER, JENNIFER",0.0,0.0,0.000000,0.524752,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.237624,0.000000,0.0
"AALGAARD, ROSS",0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ZYLAN, YVONNE",0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,1.000000,0.0
"ZYLKIN, THOMAS",0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.230769,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.076923,0.0
"ZYLSTRA, UKO",0.0,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.500000,0.0
"ZYPHUR, MICHAEL",0.0,0.0,0.000000,0.531532,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.324324,0.144144,0.0


In [258]:
lookup_subj_area_pd.to_parquet('lookup_subj_area_pd.parquet')

In [151]:
# np.save('lookup_subj_area.npy', lookup_subj_area, allow_pickle=True)